In [ ]:
import torch
import os
import evaluate
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertConfig , BertModel, DistilBertModel, BertForSequenceClassification, BertForNextSentencePrediction
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
import numpy as np

# Set the device and load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Aplication of the models
Now we gonna apply the 3 modes:
- distilbert_fact_classifier
- distilbert_fact_agrument_classifier
- distilbert_agrument_classifier

To the [`us_election_2020_2nd_presidential_debate`](Truthify/Datasets/us_debates/original/us_election_2020_2nd_presidential_debate.csv) dataset.


We also gonna do the same [`us_election_2020_trump_town_hall.csv`](Truthify/Datasets/us_debates/original/us_election_2020_trump_town_hall.csv) and [`us_election_2020_biden_town_hall.csv`](Truthify/Datasets/us_debates/original/us_election_2020_biden_town_hall.csv).

Since this dataset only contains speeches from one speakers, Trump or Biden, the goal of the analysis is to examine the frequency of specific argument types—namely, `Counterargument (Contradiction)` and `Restatement`. This analyse allow to see if speaker is actively defending their beliefs or engaging in persuasive strategies.


In [ ]:
# Load the models
model_fact_agrument_classifier = BertForSequenceClassification.from_pretrained("./models/distilbert_fact_agrument_classifier").to(device)
tokenizer_fact_agrument_classifier = BertTokenizer.from_pretrained("./models/distilbert_fact_agrument_classifier")

model_fact_classifier = BertForSequenceClassification.from_pretrained("distilbert-base-uncased").to(device)
model_fact_classifier.load_state_dict(torch.load('./models/distilbert_fact_classifier.pth', weights_only=False))
model_fact_classifier.to(device)
tokenizer_fact_classifier = BertTokenizer.from_pretrained("distilbert-base-uncased")

model_agrument_classifier = BertForSequenceClassification.from_pretrained("./models/distilbert_agrument_classifier").to(device)
tokenizer_agrument_classifier = BertTokenizer.from_pretrained("./models/distilbert_agrument_classifier")

In [ ]:
# Load datatset
second_presidential_debate=pd.read_csv(os.path.join('Datasets','us_debates','original','us_election_2020_2nd_presidential_debate.csv'))
trump_town_hall_debate=pd.read_csv(os.path.join('Datasets','us_debates','original','us_election_2020_trump_town_hall.csv'))
biden_town_hall_debate=pd.read_csv(os.path.join('Datasets','us_debates','original','us_election_2020_biden_town_hall.csv'))